### Imports and Setup

In [1]:
# Downloaded libraries
import torch
from torch import nn
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T


# Local files
from dataset_graphs import NNDataset
from models import Trainer_GCN

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Debugging Settings
torch.set_printoptions(threshold=12500)

In [3]:
# Constants
TRAINING_SPLIT = 0.8

In [4]:
# Hyperparameters
num_epoch = 8
batch_size = 8

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Data Loading

In [6]:
transform = None
# transform = T.Compose([T.ToUndirected()])

In [7]:
nndataset = NNDataset(root="../", transform=transform)

size = len(nndataset)
train_num = int(size * TRAINING_SPLIT)
test_num = size - train_num

print(
    f"Dataset loaded, {train_num} training samples and {test_num} testing samples")

Dataset loaded, 1120 training samples and 280 testing samples


In [8]:
# Preview of the Data

data = nndataset[0]
data

Data(design=[3], edge_index=[2, 40], x=[13, 503], edge_weight=[40, 1], y_node=[13, 1], y_edge=[40, 1], input_mask=[13, 1], num_nodes=13)

In [9]:
data.is_undirected()

False

In [10]:
train_loader = DataLoader(
    dataset=nndataset[:train_num], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    dataset=nndataset[train_num:], batch_size=test_num, shuffle=True)

In [11]:
print("Number of batches:", int(train_num / batch_size))

Number of batches: 140


### Loading the Model

In [12]:
model = Trainer_GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
loss_fn = nn.MSELoss()

In [13]:
model

Trainer_GCN(
  (conv1): GATConv(503, 128, heads=1)
  (conv2): GATConv(128, 128, heads=1)
  (dense_1B): Linear(in_features=128, out_features=1, bias=True)
  (dense_1W): Linear(in_features=128, out_features=1, bias=True)
)

### Training and Evaluation

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for i, data in enumerate(dataloader):
        data.to(device)

        # forward propagation
        out_w, out_b = model(data)
        loss = 0
        loss += loss_fn(out_b, data.y_node)
        loss += loss_fn(out_w, data.y_edge)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print status every n batches
        if i % 35 == 0:
            loss, current = loss.item(), i * batch_size
            print(
                f"Training Loss: {loss:>7f}  [{current:>5d}/{train_num:>5d}]")

In [15]:
def test(dataloader, model, loss_fn):
    model.eval()
    with torch.no_grad():
        data = iter(dataloader).next().to(device)

        # forward propagation
        out_w, out_b = model(data)
        loss = loss_fn(out_b, data.y_node) + loss_fn(out_w, data.y_edge)

        loss = loss.item()
        print(f"Validation Loss: {loss:>7f}")


In [16]:
# Model Training
for epoch in range(num_epoch):
    print(f"Epoch {epoch + 1} / {num_epoch}:")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)

Epoch 1 / 8:
Training Loss: 0.228955  [    0/ 1120]
Training Loss: 0.184672  [  280/ 1120]
Training Loss: 0.210227  [  560/ 1120]
Training Loss: 0.135145  [  840/ 1120]
Validation Loss: 0.122929
Epoch 2 / 8:
Training Loss: 0.128284  [    0/ 1120]
Training Loss: 0.175270  [  280/ 1120]
Training Loss: 0.137085  [  560/ 1120]
Training Loss: 0.175932  [  840/ 1120]
Validation Loss: 0.117009
Epoch 3 / 8:
Training Loss: 0.123424  [    0/ 1120]
Training Loss: 0.157154  [  280/ 1120]
Training Loss: 0.143915  [  560/ 1120]
Training Loss: 0.128124  [  840/ 1120]
Validation Loss: 0.115227
Epoch 4 / 8:
Training Loss: 0.141220  [    0/ 1120]
Training Loss: 0.147863  [  280/ 1120]
Training Loss: 0.184705  [  560/ 1120]
Training Loss: 0.154989  [  840/ 1120]
Validation Loss: 0.110405
Epoch 5 / 8:
Training Loss: 0.134065  [    0/ 1120]
Training Loss: 0.145889  [  280/ 1120]
Training Loss: 0.123261  [  560/ 1120]
Training Loss: 0.144138  [  840/ 1120]
Validation Loss: 0.108944
Epoch 6 / 8:
Training Los

### Comparison Using One Instance of Data

In [17]:
data = nndataset[0]
data = data.to(device)
data

Data(design=[3], edge_index=[2, 40], x=[13, 503], edge_weight=[40, 1], y_node=[13, 1], y_edge=[40, 1], input_mask=[13, 1], num_nodes=13)

In [22]:
data.design

[3, 8, 2]

In [18]:
out_w, out_b = model(data)

In [19]:
out_w

tensor([[ 0.0842],
        [ 0.0837],
        [ 0.0765],
        [ 0.0744],
        [ 0.0840],
        [ 0.0822],
        [ 0.0651],
        [ 0.0848],
        [-0.0181],
        [-0.0187],
        [-0.0258],
        [-0.0282],
        [-0.0183],
        [-0.0202],
        [-0.0374],
        [-0.0177],
        [ 0.0312],
        [ 0.0307],
        [ 0.0235],
        [ 0.0212],
        [ 0.0311],
        [ 0.0292],
        [ 0.0121],
        [ 0.0318],
        [-0.0037],
        [-0.0051],
        [-0.0041],
        [-0.0055],
        [-0.0113],
        [-0.0127],
        [-0.0136],
        [-0.0150],
        [-0.0038],
        [-0.0052],
        [-0.0057],
        [-0.0071],
        [-0.0228],
        [-0.0242],
        [-0.0032],
        [-0.0046]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [20]:
data.y_edge

tensor([[ 0.2277],
        [-0.5283],
        [ 0.0862],
        [-0.3806],
        [ 1.1312],
        [-0.5555],
        [-0.4393],
        [-0.4372],
        [ 1.0530],
        [ 0.4600],
        [-0.4602],
        [-0.3724],
        [-0.6324],
        [-0.3388],
        [-0.4321],
        [-0.0846],
        [ 0.3264],
        [-0.4622],
        [ 0.4405],
        [ 0.3594],
        [ 0.1728],
        [-1.1223],
        [ 0.6651],
        [ 0.2089],
        [-0.6755],
        [ 0.5243],
        [-0.4433],
        [ 0.1207],
        [ 0.5847],
        [-0.5651],
        [-0.0793],
        [-0.0928],
        [ 0.5919],
        [-0.6201],
        [-0.1842],
        [ 0.6789],
        [ 0.5201],
        [-0.5449],
        [ 0.1860],
        [ 0.1699]], device='cuda:0')

In [21]:
torch.save(model.state_dict(), "../model/model")
print("Model saved")

Model saved
